# <span style="color:darkblue"> Lecture 15 - Merging Data </span>

<font size = "5">

In the previous class we covered ...

- Aggregate Statistics
- Merge aggregate stats

In this class we will cover ...

- More database merging!
- Emphasize importance of cleaning before merging
- Database concatenation

# <span style="color:darkblue"> I. Import Libraries and Data </span>


<font size = "5">
Key libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

<font size = "5">

Read dataset on car racing circuits

- https://en.wikipedia.org/wiki/Formula_One <br>
- [See Data Source](https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)

In [36]:
results_raw  = pd.read_csv("data_raw/results.csv")
races_raw    = pd.read_csv("data_raw/races.csv")
circuits_raw = pd.read_csv("data_raw/circuits.csv")

<font size = "5">

Multi-file datasets can be visualized with an ...

- "Entity Relationship Diagram" (ERD)
- How the identifiers in each table are connected
- Complement to the "codebook"

<img src="figures/erd_f1_simple.png" alt="drawing" width="600"/>


<font size = "5">

Start by opening datasets!

- Check columns with similar names

In [3]:
# We extract all the unique values in races_raw["name"] and circuits_raw["name"]
# We use "sort_values()" to make it easier to compare the variables
# The "codebook/f1_codebook.pdf" file shows that the content is indeed different

unique_data_races    = pd.unique(races_raw["name"].sort_values())
unique_data_circuits = pd.unique(circuits_raw["name"].sort_values())


# <span style="color:darkblue"> II. Dictionaries + Renaming </span>

<font size = "5">

A dictionary is another way to store data. 

- Defined with curly brackets "{...}"
- Different fields are separated by a comma
- Assign values to a field with a colon ":"

<font size = "5">

Dictionaries + Pandas

In [4]:
# This is an example of a pandas data frame created from a dictionary
# This example illustrates the basic syntax of a dictionary

car_dictionary = {"car_model": ["Ferrari","Tesla","BMW"],
                  "year":      ["2018","2023","2022"] }

pd.DataFrame(car_dictionary)


,car_model,year
0,Ferrari,2018
1,Tesla,2023
2,BMW,2022


<font size = "5">

Rename columns with dictionaries

``` {"old_name": "new_name"} ```

In [51]:
# We first define the dictionary
# Change the pipe ".rename(...)" to rename the columns
# Dictionaries can flexibly accommodate single values or list after ":"

dict_rename_circuits = {"name": "circuit_name"}
circuits = circuits_raw.rename(columns = dict_rename_circuits)


<font size = "5">
Check that ".rename()" worked

In [16]:
# Extract the column names of the "raw" and "clean" data

print("Old List:")
print(circuits_raw.columns.values)
print("")
print("New List:")
print(circuits.columns.values)


Old List:
['circuitId' 'circuitRef' 'name' 'location' 'country' 'lat' 'lng' 'alt'
 'url']

New List:
['circuitId' 'circuitRef' 'circuit_name' 'location' 'country' 'lat' 'lng'
 'alt' 'url']


<font size = 5>

Try it yourself!

- Create a dictionary to rename "name" to "race_name"
- Rename this column in the "races_raw" dataset
- Store the output in a new dataset called "races"

In [31]:
# Write your own code
dict_race_rename = {"name": "race_name"}
races_raw = races_raw.rename(columns = dict_race_rename)
print(races_raw.columns)



Index(['raceId', 'year', 'round', 'circuitId', 'race_name', 'date', 'time',
       'url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date',
       'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time'],
      dtype='object')


# <span style="color:darkblue"> II. Merging </span>


<font size = "5">

Extracting specific columns from dataset

In [8]:
circuits[["circuitId","circuit_name"]]

,circuitId,circuit_name
0,1,Albert Park Grand Prix Circuit
1,2,Sepang International Circuit
2,3,Bahrain International Circuit
3,4,Circuit de Barcelona-Catalunya
4,5,Istanbul Park
...,...,...
72,75,Autódromo Internacional do Algarve
73,76,Autodromo Internazionale del Mugello
74,77,Jeddah Corniche Circuit
75,78,Losail International Circuit


<font size = "5">

Merge datasets

<img src="figures/merge_goal.png" alt="drawing" width="500"/>


```pd.merge(data1,data2,on,how)```

- Strive to merge only specific columns of data2
- Avoid merging all columns
- Keeping it simple gives you more control over the output

In [9]:
# The "pd.merge()" command combines the information from both datasets
# The first argument is the "primary" datasets
# The second argument is the "secondary" dataset (much include the "on" column)
# The "on" is the common variable that is used for merging
# how = "left" tells Python that the left dataset is the primary one

races_merge = pd.merge(races_raw,
                       circuits[["circuitId","circuit_name"]],
                       on = "circuitId",
                       how = "left")

In [10]:
# Another example of merging

results_merge = pd.merge(results_raw,
                         races_raw[["raceId","date"]],
                         on = "raceId",
                         how = "left")

<font size = "5">
<span style="color:red"> Common pitfall: </span> What happens if you don't rename?

In [37]:
# The following code merges the raw data
# which has the "name" column in "races_raw" and "circuits_raw"

races_merge_pitfall = pd.merge(races_raw,
                               circuits_raw[["circuitId","name"]],
                               on = "circuitId",
                               how = "left")

# Python will internally rename the columns "name_x" (for the left dataset)
# and "name_y" (for the right dataset)

print(races_merge_pitfall.columns.values)


['raceId' 'year' 'round' 'circuitId' 'name_x' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time' 'name_y']


<font size = "5">

Try it yourself!

- Rename the columns "name_x" and "name_y" <br>
in the dataset "races_merge_pitfall" to <br>
 "race_name" and "circuit_name"

$\quad$ HINT: Create a dictionary and use "pd.rename()"

In [58]:
# Write your own code
dict_rename_pitfall = {"name_x": "race_name",
                       "name_y": "circuit_name"}
races_merge_pitfall = races_merge_pitfall.rename(columns = dict_rename_pitfall)
print(races_merge_pitfall.columns.values)




['raceId' 'year' 'round' 'circuitId' 'race_name' 'date' 'time' 'url'
 'fp1_date' 'fp1_time' 'fp2_date' 'fp2_time' 'fp3_date' 'fp3_time'
 'quali_date' 'quali_time' 'sprint_date' 'sprint_time' 'circuit_name']


<font size = "5">

Try it yourself!

- Merge the column "alt", "lng", and "lat" into the races data <br>
using "pd.merge()

In [164]:
# Write your own code
selected_columns = circuits_raw[["circuitId", "alt", "lng", "lat"]]
selected_columns.rename(columns = {"circuitId": "circuitId_temp"}, inplace = True)
races_raw.merge(selected_columns, left_on = "circuitId", right_on = "circuitId_temp", indicator = True)

/Users/hp/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,...,fp3_time,quali_date,quali_time,sprint_date,sprint_time,circuitId_temp,alt,lng,lat,_merge
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,...,\N,\N,\N,\N,\N,1,10,144.9680,-37.8497,both
1,18,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,\N,\N,...,\N,\N,\N,\N,\N,1,10,144.9680,-37.8497,both
2,36,2007,1,1,Australian Grand Prix,2007-03-18,03:00:00,http://en.wikipedia.org/wiki/2007_Australian_G...,\N,\N,...,\N,\N,\N,\N,\N,1,10,144.9680,-37.8497,both
3,55,2006,3,1,Australian Grand Prix,2006-04-02,14:00:00,http://en.wikipedia.org/wiki/2006_Australian_G...,\N,\N,...,\N,\N,\N,\N,\N,1,10,144.9680,-37.8497,both
4,71,2005,1,1,Australian Grand Prix,2005-03-06,14:00:00,http://en.wikipedia.org/wiki/2005_Australian_G...,\N,\N,...,\N,\N,\N,\N,\N,1,10,144.9680,-37.8497,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1075,2022,2,77,Saudi Arabian Grand Prix,2022-03-27,17:00:00,http://en.wikipedia.org/wiki/2022_Saudi_Arabia...,2022-03-25,14:00:00,...,14:00:00,2022-03-26,17:00:00,\N,\N,77,15,39.1044,21.6319,both
1098,1099,2023,2,77,Saudi Arabian Grand Prix,2023-03-19,17:00:00,https://en.wikipedia.org/wiki/2023_Saudi_Arabi...,2023-03-17,13:30:00,...,13:30:00,2023-03-18,17:00:00,\N,\N,77,15,39.1044,21.6319,both
1099,1078,2022,5,79,Miami Grand Prix,2022-05-08,19:30:00,http://en.wikipedia.org/wiki/2022_Miami_Grand_...,2022-05-06,18:30:00,...,17:00:00,2022-05-07,20:00:00,\N,\N,79,\N,-80.2389,25.9581,both
1100,1102,2023,5,79,Miami Grand Prix,2023-05-07,19:30:00,https://en.wikipedia.org/wiki/2023_Miami_Grand...,2023-05-05,17:30:00,...,16:30:00,2023-05-06,20:00:00,\N,\N,79,\N,-80.2389,25.9581,both


# <span style="color:darkblue"> III. Concat </span>


<font size = "5">

Use ".query()" to split data into different parts

In [52]:
circuits_spain = circuits.query('country == "Spain"')
circuits_usa   = circuits.query('country == "United States" | country == "USA"')
circuits_malaysia = circuits.query('country == "Malaysia"')

<font size = "5">

Cocatenate data back together

- Useful if there are datasets split by geography...
- year, or other subgroup

In [31]:
# Works best if columns are identical
# There are also other advanced options if they are not 
# https://pandas.pydata.org/docs/reference/api/pandas.concat.html

circuits_concat = pd.concat([circuits_spain,circuits_usa])


<font size = "5">

Try it yourself!

- Concatenate the USA and Malaysia datasets



In [59]:
# Write your own code
pd.concat([circuits_usa, circuits_spain, circuits_malaysia], keys = ["usa", "spain", "malaysia"])
